In [94]:
import torch
import random

indices = torch.randint(0, 2, (2, 5))
tensor = torch.rand((2, 5, 2))
pad_e = torch.zeros((1, 1, 2))
indices

tensor([[0, 0, 0, 1, 0],
        [0, 0, 0, 1, 1]])

In [ ]:

def randomly_keep_one_with_indices(indices, pad_e, z_q):
    chosen_indices = []

    for row_idx, row in enumerate(indices):
        row_indices = []
        i = 0

        while i < len(row):
            value = row[i].item()
            
            # Detect consecutive duplicates
            j = i + 1
            while j < len(row) and row[j].item() == value:
                j += 1
            
            # Randomly select one index from the range
            random_index = random.randint(i, j - 1)
            row_indices.append(random_index)  # Track chosen index
            
            # Move to next unique value
            i = j

        chosen_indices.append(torch.tensor(row_indices, device=indices.device))
    
    # extract the embedding from z_q and pad to the length of the longest row with pad_e
    max_length = max(len(ind) for ind in chosen_indices)
    print("max_length", max_length)
    print("chosen_indices", chosen_indices)
    dummy = []
    for r, ind in enumerate(chosen_indices):
        dummy.append(z_q[r, ind].unsqueeze(0))
        print(z_q[r, ind].unsqueeze(0).shape)
        
        print("dummy", dummy[r])    
        # pad the row with pad_e
        pad_len = max_length - len(ind)
        print("pad_len", pad_len)
        if pad_len > 0:
            pad = pad_e.repeat(1, pad_len,1)
            print("pad", pad.shape)
            dummy[r] = torch.cat([dummy[r], pad], dim=1)
    
    # concatenate the rows
    z_q_disc = torch.cat(dummy, dim=0)

    return z_q_disc

out = randomly_keep_one_with_indices(indices, pad_e, tensor)

max_length 3
chosen_indices [tensor([2, 3, 4]), tensor([0, 3])]
torch.Size([1, 3, 2])
dummy tensor([[0.0274, 0.0332],
        [0.0860, 0.8944],
        [0.3660, 0.7036]])
pad_len 0
torch.Size([1, 2, 2])
dummy tensor([[0.2411, 0.3674],
        [0.3298, 0.8448]])
pad_len 1
pad torch.Size([1, 1, 2])


RuntimeError: Tensors must have same number of dimensions: got 2 and 3

In [110]:
tensor[1]

tensor([[0.2411, 0.3674],
        [0.8927, 0.3168],
        [0.4515, 0.9828],
        [0.3298, 0.8448],
        [0.2756, 0.1185]])

In [109]:
out

tensor([[[0.0274, 0.0332],
         [0.0860, 0.8944],
         [0.3660, 0.7036]],

        [[0.8927, 0.3168],
         [0.2756, 0.1185],
         [0.0000, 0.0000]]])